# 🆔 マスターIDリスト取得ツール (Web Crawler)
Netkeibaの開催カレンダーを巡回し、指定期間内の**全てのレースID**を取得して `race_ids.csv` (JRA) / `race_ids_nar.csv` (NAR) を作成します。
これらが「マスターリスト」となり、スクレイピングツールはこのリストにあるが手元のCSVにないデータのみを取得するようになります。

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 設定
DATA_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'
JRA_ID_CSV = 'race_ids.csv'
NAR_ID_CSV = 'race_ids_nar.csv'

START_YEAR = 2020
END_YEAR = 2026


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
from tqdm.auto import tqdm
from datetime import date, timedelta

def fetch_race_ids(mode='JRA', start_year=2020, end_year=2026):
    print(f"\n🚀 {mode} Race ID Fetching ({start_year}-{end_year})...")
    
    # Base URLs
    # JRA: race.netkeiba.com
    # NAR: nar.netkeiba.com
    base_domain = "race.netkeiba.com" if mode == 'JRA' else "nar.netkeiba.com"
    save_csv = JRA_ID_CSV if mode == 'JRA' else NAR_ID_CSV
    save_path = os.path.join(DATA_DIR, save_csv)
    
    all_ids = set()
    
    # If file exists, load it first to append/update? 
    # User request implies "Master List", so maybe overwrite or merge?
    # Let's load existing to avoid duplicates, but crawling should be authoritative.
    if os.path.exists(save_path):
        try:
            existing_df = pd.read_csv(save_path, dtype=str)
            if 'race_id' in existing_df.columns:
                all_ids = set(existing_df['race_id'].unique())
                print(f"  Loaded {len(all_ids)} existing IDs from {save_csv}")
        except: pass

    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    
    for year in range(start_year, end_year + 1):
        print(f"  📅 Processing {year}...")
        for month in range(1, 13):
            # Calendar Page
            # https://race.netkeiba.com/top/calendar.html?year=2025&month=1
            cal_url = f"https://{base_domain}/top/calendar.html?year={year}&month={month}"
            try:
                time.sleep(0.5)
                resp = session.get(cal_url, headers=headers, timeout=10)
                resp.encoding = 'EUC-JP'
                soup = BeautifulSoup(resp.text, 'html.parser')
                
                # Find days with links
                # Output format: <td class="RaceCell"><a href="../top/race_list.html?kaisai_date=20250105">...</a></td>
                # Links: ../top/race_list.html?kaisai_date=YYYYMMDD
                day_links = soup.select('a[href*="race_list.html?kaisai_date="]')
                dates = []
                for link in day_links:
                    href = link.get('href')
                    m = re.search(r'kaisai_date=(\d{8})', href)
                    if m:
                        dates.append(m.group(1))
                
                dates = sorted(list(set(dates)))
                if not dates: continue
                
                # Iterate Days
                for d_str in tqdm(dates, desc=f"{year}/{month:02}", leave=False):
                    # Race List Page
                    # https://race.netkeiba.com/top/race_list.html?kaisai_date=20250105
                    list_url = f"https://{base_domain}/top/race_list.html?kaisai_date={d_str}"
                    try:
                        time.sleep(0.3)
                        r_resp = session.get(list_url, headers=headers, timeout=10)
                        r_resp.encoding = 'EUC-JP'
                        r_soup = BeautifulSoup(r_resp.text, 'html.parser')
                        
                        # Find Race Links
                        # <a href="../race/result.html?race_id=202506010101">...</a>
                        # or shutuba.html (if future)
                        race_links = r_soup.select('a[href*="race_id="]')
                        
                        day_ids = set()
                        for r_link in race_links:
                            r_href = r_link.get('href')
                            rm = re.search(r'race_id=(\d+)', r_href)
                            if rm:
                                rid = rm.group(1)
                                # JRA IDs usually 12 digits. NAR can be 12?
                                # Validate length just in case? 
                                # Netkeiba unified ID is usually 12 digits (YYYY PP KK DD RR)
                                if len(rid) == 12:
                                    day_ids.add(rid)
                        
                        all_ids.update(day_ids)
                        
                    except Exception as e:
                         print(f"    Error fetching day {d_str}: {e}")
                         
            except Exception as e:
                print(f"    Error fetching calendar {year}/{month}: {e}")
    
    # Save
    print(f"\n💾 Saving {len(all_ids)} IDs to {save_path}...")
    df_out = pd.DataFrame({'race_id': sorted(list(all_ids))})
    df_out.to_csv(save_path, index=False)
    print("✅ Done.")



In [ ]:
# 実行
if os.path.exists(DATA_DIR):
    try:
        fetch_race_ids(mode='JRA', start_year=START_YEAR, end_year=END_YEAR)
    except Exception as e: print(f"JRA Error: {e}")
    
    try:
        fetch_race_ids(mode='NAR', start_year=START_YEAR, end_year=END_YEAR)
    except Exception as e: print(f"NAR Error: {e}")
else:
    print(f"❌ ディレクトリが見つかりません: {DATA_DIR}")
